In [1]:
!nvidia-smi

Fri Oct 29 18:36:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:8B:00.0 Off |                    0 |
| N/A   34C    P0    36W / 300W |   1385MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
DATA_DIR = '../data/'

In [2]:
import pandas as pd
import numpy as np

In [3]:
!ls ../data

chronom_test.csv   gas_test.csv     plavki_train.csv	   sip_train.csv
chronom_train.csv  gas_train.csv    produv_test.csv	   target_train.csv
chugun_test.csv    lom_test.csv     produv_train.csv
chugun_train.csv   lom_train.csv    sample_submission.csv
datka.zip	   plavki_test.csv  sip_test.csv


In [4]:
df_train_static = pd.read_csv(DATA_DIR + 'plavki_train.csv').drop_duplicates('NPLV').reset_index(drop=True)
df_test_static = pd.read_csv(DATA_DIR + 'plavki_test.csv')

target = pd.read_csv(DATA_DIR + 'target_train.csv')
df_train_static = df_train_static.merge(target, on='NPLV', how='left')

for plavki in [df_train_static, df_test_static]:
    plavki['plavka_VR_NACH'] = pd.to_datetime(plavki['plavka_VR_NACH'])
    plavki['plavka_VR_KON'] = pd.to_datetime(plavki['plavka_VR_KON'])
    plavki['plav_seconds'] = (((plavki.plavka_VR_KON.values - plavki.plavka_VR_NACH.values))*1e-9).astype(int)
df_train_static.head()

,NPLV,plavka_VR_NACH,plavka_VR_KON,plavka_NMZ,plavka_NAPR_ZAD,plavka_STFUT,plavka_TIPE_FUR,plavka_ST_FURM,plavka_TIPE_GOL,plavka_ST_GOL,TST,C,plav_seconds
0,510008,2021-01-01 03:08:11,2021-01-01 03:51:10,С255,МНЛЗ,971,цилиндрическая,11,5 сопловая,11,1690,0.060,2579
1,510009,2021-01-01 04:00:44,2021-01-01 05:07:28,С255,МНЛЗ,972,цилиндрическая,12,5 сопловая,12,1683,0.097,4004
2,510010,2021-01-01 05:12:29,2021-01-01 06:00:53,Ст3пс/Э,Изл,973,цилиндрическая,13,5 сопловая,13,1662,0.091,2904
3,510011,2021-01-01 06:13:48,2021-01-01 07:08:39,Св-08А.z02,Изл,974,цилиндрическая,14,5 сопловая,14,1609,0.410,3291
4,510012,2021-01-01 07:13:44,2021-01-01 08:01:59,SC2M/ЭТ,МНЛС,975,цилиндрическая,15,5 сопловая,15,1682,0.120,2895


In [6]:
static_cat_feats = ['plavka_NAPR_ZAD', 'plavka_TIPE_FUR', 'plavka_TIPE_GOL', 'plavka_NMZ']
static_num_feats = ['plavka_STFUT','plavka_ST_FURM','plav_seconds','plavka_ST_GOL']
target_fts = ['TST','C']
ts_cat_feats = ['NMSYP']
ts_num_feats = ['V', 'T', 'O2', 'N2', 'H2', 'CO2', 'CO', 'AR',
       'T фурмы 1', 'T фурмы 2', 'O2_pressure', 'RAS', 'POL', 'VDSYP',
       'VSSYP', 'VES_ch', 'T_ch', 'SI_ch', 'MN_ch', 'S_ch', 'P_ch', 'CR_ch',
       'NI_ch', 'CU_ch', 'V_ch', 'TI_ch']

In [7]:
def make_dataset(data_dir, postfix = 'train'):
    plavki = pd.read_csv(data_dir + 'plavki_'+postfix + '.csv')
    plavki['plavka_VR_NACH'] = pd.to_datetime(plavki['plavka_VR_NACH'])
    plavki['plavka_VR_KON'] = pd.to_datetime(plavki['plavka_VR_KON'])
    gas = pd.read_csv(data_dir + 'gas_'+postfix + '.csv')
    gas['Time'] = pd.to_datetime(gas['Time'])
    produv = pd.read_csv(data_dir + 'produv_'+postfix + '.csv')
    produv['SEC'] = pd.to_datetime(produv['SEC'])
    sip = pd.read_csv(data_dir + 'sip_'+postfix + '.csv')
    sip['DAT_OTD'] = pd.to_datetime(sip['DAT_OTD'])
    chugun = pd.read_csv(data_dir + 'chugun_'+postfix + '.csv')
    chugun.columns = ['NPLV'] + [x+'_ch' for x in chugun.columns.tolist()[1:-1]] + ['DATA_ZAMERA']
    
    dfs = []
    for g,df in gas.groupby('NPLV'):
        print(g)
        p = plavki.loc[plavki.NPLV == g]
        p['plav_seconds'] = int(((p.plavka_VR_KON.values[0] - p.plavka_VR_NACH.values[0])).item()*1e-9)
        df1 = df.loc[df.Time <= p.plavka_VR_KON.values[0]]
        ppdf = produv.loc[(produv.NPLV == g) & (produv.SEC <= p.plavka_VR_KON.values[0])].drop(columns=['NPLV'])
        df1 = pd.merge_asof(df1, ppdf, left_on='Time', right_on='SEC').drop(columns=['SEC'])
        psip = sip.loc[(produv.NPLV == g) & (sip.DAT_OTD <= p.plavka_VR_KON.values[0])].drop(columns=['NPLV'])
        df1 = pd.merge_asof(df1, psip, left_on='Time', right_on='DAT_OTD').drop(columns=['DAT_OTD'])
        df1 = df1.merge(chugun, on='NPLV', how='left')
        df1 = df1.drop(columns = ['DATA_ZAMERA'])
        dfs.append(df1)
        #if len(dfs) > 100:
        #    break
        
    df = pd.concat(dfs, ignore_index = True).reset_index(drop=True)
    df[ts_num_feats] = df[ts_num_feats].interpolate()
    return df

df_train = make_dataset(DATA_DIR, 'train')
df_test = make_dataset(DATA_DIR, 'test')

510008


/tmp/ipykernel_287456/1568610564.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p['plav_seconds'] = int(((p.plavka_VR_KON.values[0] - p.plavka_VR_NACH.values[0])).item()*1e-9)


510009
510010
510011
510012
510013
510014
510015
510016
510017
510018
510019
510020
510021
510022
510023
510024
510026
510027
510028
510030
510031
510032
510033
510037
510040
510043
510044
510045
510047
510048
510049
510050
510051
510052
510053
510055
510057
510058
510059
510060
510061
510062
510063
510064
510066
510067
510068
510069
510070
510071
510072
510073
510074
510075
510076
510077
510078
510079
510080
510081
510082
510083
510084
510087
510088
510089
510090
510091
510092
510093
510094
510095
510096
510097
510098
510099
510100
510101
510102
510103
510104
510105
510106
510107
510108
510109
510110
510111
510112
510114
510115
510116
510117
510118
510119
510120
510121
510122
510124
510125
510126
510128
510130
510131
510132
510133
510134
510135
510136
510137
510138
510139
510140
510141
510142
510143
510144
510145
510146
510147
510148
510149
510151
510152
510153
510156
510157
510158
510159
510160
510161
510163
510164
510165
510166
510167
510168
510169
510170
510171
510172
510174
510175

511321
511322
511323
511324
511325
511326
511327
511328
511329
511330
511331
511332
511333
511334
511335
511336
511337
511338
511339
511341
511342
511343
511344
511345
511346
511347
511348
511349
511350
511351
511352
511353
511354
511355
511356
511357
511358
511359
511360
511361
511362
511363
511364
511365
511366
511367
511368
511369
511370
511371
511372
511375
511378
511379
511380
511382
511383
511384
511385
511386
511387
511388
511389
511390
511391
511392
511393
511394
511395
511396
511397
511398
511399
511400
511401
511402
511403
511404
511405
511406
511407
511408
511409
511410
511411
511412
511413
511414
511415
511417
511418
511419
511420
511422
511423
511424
511425
511426
511427
511429
511430
511431
511432
511433
511434
511435
511436
511437
511438
511439
511440
511441
511442
511443
511444
511445
511446
511447
511448
511449
511450
511451
511452
511454
511455
511456
511457
511458
511462
511463
511464
511465
511466
511467
511468
511469
511470
511471
511472
511473
511474
511475
511476

512698
512699
512700
512701
512702
512704
512705
512706
512707
512708
512709
512710
512711
512712
512713
512714
512715
512716
512718
512719
512720
512721
512723
512725
512726
512727
512728
512729
512730
512731
512736
512737
512738
512739
512740
512741
512742
512743
512744
512745
512746
512747
512748
512749
512750
512751
512752
512753
512754
512755
512756
512757
512758
512759
512760
512761
512762
512763
512764
512765
512768
512769
512775
512776
512777
512778
512782
512788
512793
512796
512797
512799
512800
512801
512802
512803
512804
512805
512806
512807
512813
512814
512815
512819
512821
512823
512824
512825
512826
512829
512830
512832
512833
512834
512836
512838
512839
512840
512841
512842
512843
512844
512845
512846
512847
512849
512850
512851
512852
512853
512854
512855
512856
512857
512858
512859
512861
512862
512863
512864
512865
512866
512867
512868
512869
512870
512871
512872
512873
512874
512876
512878
512882
512884
512885
512887
512888
512891
512892
512893
512894
512897
512898

In [8]:
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv', index=False)

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
le.fit(df_train_static['plavka_NAPR_ZAD'])
df_train_static['plavka_NAPR_ZAD'] = le.transform(df_train_static['plavka_NAPR_ZAD'])
df_test_static['plavka_NAPR_ZAD'] = le.transform(df_test_static['plavka_NAPR_ZAD'])

le = LabelEncoder()
le.fit(df_train_static['plavka_TIPE_FUR'])
df_train_static['plavka_TIPE_FUR'] = le.transform(df_train_static['plavka_TIPE_FUR'])
df_test_static['plavka_TIPE_FUR'] = le.transform(df_test_static['plavka_TIPE_FUR'])

le = LabelEncoder()
plavka_nmz = set(df_train_static['plavka_NMZ'].unique().tolist()) & set(df_test_static['plavka_NMZ'].unique().tolist())
df_train_static.loc[~df_train_static['plavka_NMZ'].isin(plavka_nmz), 'plavka_NMZ'] = 'None'
df_test_static.loc[~df_test_static['plavka_NMZ'].isin(plavka_nmz), 'plavka_NMZ'] = 'None'
le.fit(df_train_static['plavka_NMZ'])
df_train_static['plavka_NMZ'] = le.transform(df_train_static['plavka_NMZ'])
df_test_static['plavka_NMZ'] = le.transform(df_test_static['plavka_NMZ'])

le = LabelEncoder()
plavka_nmz = set(df_train_static['plavka_TIPE_GOL'].unique().tolist()) & set(df_test_static['plavka_TIPE_GOL'].unique().tolist())
df_train_static.loc[~df_train_static['plavka_TIPE_GOL'].isin(plavka_nmz), 'plavka_TIPE_GOL'] = 'None'
df_test_static.loc[~df_test_static['plavka_TIPE_GOL'].isin(plavka_nmz), 'plavka_TIPE_GOL'] = 'None'
le.fit(df_train_static['plavka_TIPE_GOL'])
df_train_static['plavka_TIPE_GOL'] = le.transform(df_train_static['plavka_TIPE_GOL'])
df_test_static['plavka_TIPE_GOL'] = le.transform(df_test_static['plavka_TIPE_GOL'])

ss = StandardScaler()
ss.fit(df_train_static[static_num_feats])
df_train_static[static_num_feats] = ss.transform(df_train_static[static_num_feats])
df_test_static[static_num_feats] = ss.transform(df_test_static[static_num_feats])

target_scaler = StandardScaler()
df_train_static[target_fts] = target_scaler.fit_transform(df_train_static[target_fts])

le = LabelEncoder()
plavka_nmz = set(df_train['NMSYP'].unique().tolist()) & set(df_test['NMSYP'].unique().tolist())
df_train.loc[~df_train['NMSYP'].isin(plavka_nmz), 'NMSYP'] = 'None'
df_test.loc[~df_test['NMSYP'].isin(plavka_nmz), 'NMSYP'] = 'None'
le.fit(df_train['NMSYP'])
df_train['NMSYP'] = le.transform(df_train['NMSYP'])
df_test['NMSYP'] = le.transform(df_test['NMSYP'])

ss = StandardScaler()
ss.fit(df_train[ts_num_feats])
df_train[ts_num_feats] = ss.transform(df_train[ts_num_feats])
df_test[ts_num_feats] = ss.transform(df_test[ts_num_feats])

In [10]:
df_train

,NPLV,Time,V,T,O2,N2,H2,CO2,CO,AR,...,T_ch,SI_ch,MN_ch,S_ch,P_ch,CR_ch,NI_ch,CU_ch,V_ch,TI_ch
0,510008,2021-01-01 03:08:11.437,0.737329,-0.530477,0.944704,0.840958,-0.437176,-1.321086,-0.718117,0.384588,...,-0.018188,-0.184487,-0.343679,1.172270,0.499569,1.421682,0.289499,0.921668,1.151996,0.290510
1,510008,2021-01-01 03:08:12.437,0.737329,-0.530477,0.945712,0.841236,-0.436921,-1.322805,-0.718154,0.383246,...,-0.018188,-0.184487,-0.343679,1.172270,0.499569,1.421682,0.289499,0.921668,1.151996,0.290510
2,510008,2021-01-01 03:08:13.437,0.747323,-0.532999,0.946720,0.841513,-0.436666,-1.324525,-0.718192,0.381905,...,-0.018188,-0.184487,-0.343679,1.172270,0.499569,1.421682,0.289499,0.921668,1.151996,0.290510
3,510008,2021-01-01 03:08:14.437,0.757315,-0.534261,0.947728,0.841791,-0.436411,-1.326244,-0.718229,0.380564,...,-0.018188,-0.184487,-0.343679,1.172270,0.499569,1.421682,0.289499,0.921668,1.151996,0.290510
4,510008,2021-01-01 03:08:15.437,0.747323,-0.538044,0.948736,0.842068,-0.436156,-1.327964,-0.718267,0.379222,...,-0.018188,-0.184487,-0.343679,1.172270,0.499569,1.421682,0.289499,0.921668,1.151996,0.290510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466980,512322,2021-04-26 18:48:35.437,-0.287989,-1.055164,-0.799696,1.611167,-0.434369,-1.334230,-0.718529,-0.267209,...,0.273195,0.110595,0.630001,0.597517,0.772881,0.147417,0.289499,0.921668,0.209882,-0.208852
6466981,512322,2021-04-26 18:48:36.437,-0.319581,-1.055164,-0.714720,1.574657,-0.446873,-1.335384,-0.718529,-0.176203,...,0.273195,0.110595,0.630001,0.597517,0.772881,0.147417,0.289499,0.921668,0.209882,-0.208852
6466982,512322,2021-04-26 18:48:37.437,-0.309045,-1.055164,-0.604872,1.526879,-0.434369,-1.336538,-0.718529,-0.176203,...,0.273195,0.110595,0.630001,0.597517,0.772881,0.147417,0.289499,0.921668,0.209882,-0.208852
6466983,512322,2021-04-26 18:48:38.437,-0.340669,-1.053903,-0.643215,1.543556,-0.434369,-1.335384,-0.718529,-0.267209,...,0.273195,0.110595,0.630001,0.597517,0.772881,0.147417,0.289499,0.921668,0.209882,-0.208852


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class EvrazDatasetV(Dataset):
    def __init__(self, feat_df, static_train_df):
        self.feat_df = feat_df.copy()
        self.static_train_df = static_train_df.copy()
        
        self.nlmv =  self.feat_df.NPLV.unique()

    def __len__(self):
        return len(self.nlmv)

    def __getitem__(self, idx):    
        nlmv = self.nlmv[idx]
        return {"static_cat": self.static_train_df.loc[self.static_train_df.NPLV == nlmv, static_cat_feats].fillna(0).values[0],
                "static_numeric": self.static_train_df.loc[self.static_train_df.NPLV == nlmv, static_num_feats].fillna(0).values[0],
                "ts_cat": self.feat_df.loc[self.feat_df.NPLV == nlmv, ts_cat_feats].fillna(0).values,
                "ts_numeric": self.feat_df.loc[self.feat_df.NPLV == nlmv, ts_num_feats].fillna(0).values,
                "y": self.static_train_df.loc[self.static_train_df.NPLV == nlmv, target_fts].values[0]
               }

In [68]:
class AttentionWeightedAverage(nn.Module):
    def __init__(self, hidden_dim: int, return_attention: bool = False):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.return_attention = return_attention
        
        self.attention_vector = nn.Parameter(
            torch.empty(self.hidden_dim, dtype=torch.float32),
            requires_grad=True,
        )
        nn.init.xavier_normal_(self.attention_vector.unsqueeze(-1))

    def forward(
        self, x: torch.Tensor, mask: torch.Tensor
    ):
        logits = x.matmul(self.attention_vector)
        ai = (logits - logits.max()).exp()

        ai = ai * mask
        att_weights = ai / (ai.sum(dim=1, keepdim=True) + 1e-12)
        weighted_input = x * att_weights.unsqueeze(-1)
        output = weighted_input.sum(dim=1)

        if self.return_attention:
            return output, att_weights
        else:
            return output, None
        

CONV_FEAT_SIZE = 16
CONV_DELS = [1,2,4,8,16,32]

class EvrazModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embs0 = nn.Embedding(3, 3)
        self.embs1 = nn.Embedding(2, 3)
        self.embs2 = nn.Embedding(4, 3)
        self.embs3 = nn.Embedding(29, 3)
        self.embs4 = nn.Embedding(4, 3)
        
        self.convs = nn.ModuleList([
            nn.Sequential(
                torch.nn.Conv1d(self.get_conv_size(i), CONV_FEAT_SIZE, 2, stride=1, 
                                padding=0, dilation=d, bias=False),
                nn.BatchNorm1d(CONV_FEAT_SIZE),
                nn.SiLU()
            )
            for i,d in enumerate(CONV_DELS)
        ])
        self.conv_attn = AttentionWeightedAverage(CONV_FEAT_SIZE)
                
        self.head = nn.Sequential(
            nn.LayerNorm(CONV_FEAT_SIZE+len(static_num_feats)+3*4),
            nn.Linear(CONV_FEAT_SIZE+len(static_num_feats)+3*4, 8),
            nn.LayerNorm(8),
            nn.SiLU(),
            nn.Linear(8, 8),
            nn.LayerNorm(8),
            nn.SiLU(),
            nn.Linear(8,2)
        )
        
    def get_conv_size(self, i):
        if i == 0:
            return len(ts_num_feats)+3
        return CONV_FEAT_SIZE
    
    def forward(self, static_cat, static_numeric, ts_cat, ts_numeric):
        x0 = self.embs0(static_cat[:,0])
        x1 = self.embs1(static_cat[:,1])
        x2 = self.embs2(static_cat[:,2])
        x3 = self.embs3(static_cat[:,3])
        x4 = static_numeric
        
        y0 = self.embs4(ts_cat[:,:,0])
        y = torch.cat([y0,ts_numeric], dim=-1).transpose(1,2)
        
        for m in self.convs:
            y = m(y)
        
        y = y.transpose(1,2)
        y = self.conv_attn(y, torch.ones((y.shape[0], y.shape[1])).to(y.device))[0]
        
        x = torch.cat([y,x0,x1,x2,x3,x4], dim=-1)
        
        x[torch.isnan(x)] = 0
        x[torch.isinf(x)] = 0
        
        return self.head(x)

In [21]:
model = EvrazModel().to(device)

In [66]:
ds = EvrazDatasetV(df_train, df_train_static)
dl = DataLoader(ds, shuffle=False, batch_size=1, num_workers=0)

In [67]:
for x in dl:
    out = model(x["static_cat"].long().to(device),
          x['static_numeric'].float().to(device),
          x['ts_cat'].long().to(device),
          x['ts_numeric'].float().to(device)
         )
    break

In [68]:
out

tensor([[-0.2516,  0.0375]], grad_fn=<AddmmBackward0>)

In [14]:
def metric(answers, user_csv):
    delta_c = np.abs(np.array(answers[:,1]) - np.array(user_csv[:,1]))
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(np.array(answers[:,0]) - np.array(user_csv[:,0]))
    hit_rate_t = np.int64(delta_t < 20)

    N = np.size(answers[:,0])

    return np.sum(hit_rate_c + hit_rate_t) / 2 / N

In [36]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=239)

nplvs = df_train_static.NPLV.unique()

for vipl_train, vipl_valid in kf.split(nplvs):
    break
nplv_train = [nplvs[x] for x in vipl_train if nplvs[x] not in [511156,512299]]
nplv_valid = [nplvs[x] for x in vipl_valid if nplvs[x] not in [511156,512299]]

In [69]:
train_ds = EvrazDatasetV(df_train.loc[df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=1, num_workers=0)
valid_ds = EvrazDatasetV(df_train.loc[~df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[~df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
valid_dl = DataLoader(valid_ds, shuffle=False, batch_size=1, num_workers=0)

model = EvrazModel().to(device)
criterion1 = nn.L1Loss()
criterion2 = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.0)


model.train();
optimizer.zero_grad()

best_metric = 0
for epoch in range(300):
    losses = []
    for x in train_dl:
        out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
             )
        loss1 = criterion1(out[:,0], x['y'].float().to(device)[:,0])
        loss2 = criterion2(out[:,1], x['y'].float().to(device)[:,1])
        loss = loss1*0.5+loss2
        #losses.append(loss.item())
        #if len(losses) == 100:
        #    print(torch.mean(torch.tensor(losses)))
        #    losses = []
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad()
    
    model.eval()
    outs, ys = [], []
    with torch.no_grad():
        for x in valid_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            ys.append(x['y'].numpy())
            outs.append(out.detach().cpu().numpy())
    
    outs = np.concatenate(outs, axis=0)
    ys = np.concatenate(ys, axis=0)
    
    outs = target_scaler.inverse_transform(outs)
    ys = target_scaler.inverse_transform(ys)
    
    m = metric(outs, ys)
    print(epoch, m)
    if m > best_metric:
        torch.save(model.state_dict(), 'model0.pth')
        best_metric = m
    model.train()
    optimizer.zero_grad()

0 0.5422705314009661
3 0.46980676328502413
4 0.48792270531400966
5 0.501207729468599
6 0.48792270531400966
7 0.45410628019323673
8 0.4758454106280193
9 0.48429951690821255
10 0.4504830917874396
11 0.4963768115942029
12 0.48188405797101447
13 0.4758454106280193
14 0.4830917874396135
15 0.4855072463768116
16 0.46859903381642515
17 0.4504830917874396
18 0.3864734299516908
19 0.46980676328502413
20 0.45410628019323673
21 0.46980676328502413
22 0.45169082125603865
23 0.4444444444444444
24 0.4601449275362319
25 0.4227053140096618
26 0.4528985507246377
27 0.4384057971014493
28 0.46497584541062803
29 0.41183574879227053
30 0.42632850241545894
31 0.42995169082125606
32 0.4251207729468599
33 0.40458937198067635
34 0.43719806763285024
35 0.4601449275362319
36 0.4033816425120773
37 0.3973429951690821
38 0.40700483091787437
39 0.4166666666666667
40 0.4504830917874396
41 0.4214975845410628
42 0.4227053140096618
43 0.3864734299516908
44 0.29589371980676327
45 0.42028985507246375
46 0.3695652173913043

In [58]:
i = 0
for vipl_train, vipl_valid in kf.split(nplvs):
    if i == 1:
        break
    i += 1
    
nplv_train = [nplvs[x] for x in vipl_train if nplvs[x] not in [511156,512299]]
nplv_valid = [nplvs[x] for x in vipl_valid if nplvs[x] not in [511156,512299]]

train_ds = EvrazDatasetV(df_train.loc[df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=1, num_workers=0)
valid_ds = EvrazDatasetV(df_train.loc[~df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[~df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
valid_dl = DataLoader(valid_ds, shuffle=False, batch_size=1, num_workers=0)

model = EvrazModel().to(device)
criterion1 = nn.L1Loss()
criterion2 = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)

model.train();
optimizer.zero_grad()

best_metric = 0
for epoch in range(100):
    losses = []
    for x in train_dl:
        out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
             )
        loss1 = criterion1(out[:,0], x['y'].float().to(device)[:,0])
        loss2 = criterion2(out[:,1], x['y'].float().to(device)[:,1])
        loss = loss1*0.5+loss2
        #losses.append(loss.item())
        #if len(losses) == 100:
        #    print(torch.mean(torch.tensor(losses)))
        #    losses = []
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad()
    
    model.eval()
    outs, ys = [], []
    with torch.no_grad():
        for x in valid_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            ys.append(x['y'].numpy())
            outs.append(out.detach().cpu().numpy())
    
    outs = np.concatenate(outs, axis=0)
    ys = np.concatenate(ys, axis=0)
    
    outs = target_scaler.inverse_transform(outs)
    ys = target_scaler.inverse_transform(ys)
    
    m = metric(outs, ys)
    print(epoch, m)
    if m > best_metric:
        torch.save(model.state_dict(), 'model1.pth')
        best_metric = m
    model.train()
    optimizer.zero_grad()

0 0.5433734939759036
1 0.5433734939759036
2 0.5566265060240964
3 0.5662650602409639
4 0.5481927710843374
5 0.5650602409638554
6 0.5650602409638554


KeyboardInterrupt: 

In [62]:
i = 0
for vipl_train, vipl_valid in kf.split(nplvs):
    if i == 2:
        break
    i += 1
    
nplv_train = [nplvs[x] for x in vipl_train if nplvs[x] not in [511156,512299]]
nplv_valid = [nplvs[x] for x in vipl_valid if nplvs[x] not in [511156,512299]]

train_ds = EvrazDatasetV(df_train.loc[df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=1, num_workers=0)
valid_ds = EvrazDatasetV(df_train.loc[~df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[~df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
valid_dl = DataLoader(valid_ds, shuffle=False, batch_size=1, num_workers=0)

model = EvrazModel().to(device)
criterion1 = nn.L1Loss()
criterion2 = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)

model.train();
optimizer.zero_grad()

best_metric = 0
for epoch in range(100):
    losses = []
    for x in train_dl:
        out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
             )
        loss1 = criterion1(out[:,0], x['y'].float().to(device)[:,0])
        loss2 = criterion2(out[:,1], x['y'].float().to(device)[:,1])
        loss = loss1*0.5+loss2
        #losses.append(loss.item())
        #if len(losses) == 100:
        #    print(torch.mean(torch.tensor(losses)))
        #    losses = []
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad()
    
    model.eval()
    outs, ys = [], []
    with torch.no_grad():
        for x in valid_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            ys.append(x['y'].numpy())
            outs.append(out.detach().cpu().numpy())
    
    outs = np.concatenate(outs, axis=0)
    ys = np.concatenate(ys, axis=0)
    
    outs = target_scaler.inverse_transform(outs)
    ys = target_scaler.inverse_transform(ys)
    
    m = metric(outs, ys)
    print(epoch, m)
    if m > best_metric:
        torch.save(model.state_dict(), 'model2.pth')
        best_metric = m
    model.train()
    optimizer.zero_grad()

0 0.5205314009661836
1 0.5289855072463768
2 0.5289855072463768
3 0.532608695652174
4 0.533816425120773
5 0.532608695652174
6 0.5072463768115942


KeyboardInterrupt: 

In [63]:
i = 0
for vipl_train, vipl_valid in kf.split(nplvs):
    if i == 3:
        break
    i += 1
    
nplv_train = [nplvs[x] for x in vipl_train if nplvs[x] not in [511156,512299]]
nplv_valid = [nplvs[x] for x in vipl_valid if nplvs[x] not in [511156,512299]]

train_ds = EvrazDatasetV(df_train.loc[df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=1, num_workers=0)
valid_ds = EvrazDatasetV(df_train.loc[~df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[~df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
valid_dl = DataLoader(valid_ds, shuffle=False, batch_size=1, num_workers=0)

model = EvrazModel().to(device)
criterion1 = nn.L1Loss()
criterion2 = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)

model.train();
optimizer.zero_grad()

best_metric = 0
for epoch in range(100):
    losses = []
    for x in train_dl:
        out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
             )
        loss1 = criterion1(out[:,0], x['y'].float().to(device)[:,0])
        loss2 = criterion2(out[:,1], x['y'].float().to(device)[:,1])
        loss = loss1*0.5+loss2
        #losses.append(loss.item())
        #if len(losses) == 100:
        #    print(torch.mean(torch.tensor(losses)))
        #    losses = []
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad()
    
    model.eval()
    outs, ys = [], []
    with torch.no_grad():
        for x in valid_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            ys.append(x['y'].numpy())
            outs.append(out.detach().cpu().numpy())
    
    outs = np.concatenate(outs, axis=0)
    ys = np.concatenate(ys, axis=0)
    
    outs = target_scaler.inverse_transform(outs)
    ys = target_scaler.inverse_transform(ys)
    
    m = metric(outs, ys)
    print(epoch, m)
    if m > best_metric:
        torch.save(model.state_dict(), 'model3.pth')
        best_metric = m
    model.train()
    optimizer.zero_grad()

0 0.5531400966183575
1 0.5591787439613527
2 0.5881642512077294
3 0.5664251207729468
4 0.5797101449275363
5 0.5555555555555556


KeyboardInterrupt: 

In [64]:
i = 0
for vipl_train, vipl_valid in kf.split(nplvs):
    if i == 4:
        break
    i += 1
    
nplv_train = [nplvs[x] for x in vipl_train if nplvs[x] not in [511156,512299]]
nplv_valid = [nplvs[x] for x in vipl_valid if nplvs[x] not in [511156,512299]]

train_ds = EvrazDatasetV(df_train.loc[df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=1, num_workers=0)
valid_ds = EvrazDatasetV(df_train.loc[~df_train.NPLV.isin(nplv_train)].reset_index(drop=True), 
                         df_train_static.loc[~df_train_static.NPLV.isin(nplv_train)].reset_index(drop=True))
valid_dl = DataLoader(valid_ds, shuffle=False, batch_size=1, num_workers=0)

model = EvrazModel().to(device)
criterion1 = nn.L1Loss()
criterion2 = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)

model.train();
optimizer.zero_grad()

best_metric = 0
for epoch in range(100):
    losses = []
    for x in train_dl:
        out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
             )
        loss1 = criterion1(out[:,0], x['y'].float().to(device)[:,0])
        loss2 = criterion2(out[:,1], x['y'].float().to(device)[:,1])
        loss = loss1*0.5+loss2
        #losses.append(loss.item())
        #if len(losses) == 100:
        #    print(torch.mean(torch.tensor(losses)))
        #    losses = []
        loss.backward()
        optimizer.step() 
        optimizer.zero_grad()
    
    model.eval()
    outs, ys = [], []
    with torch.no_grad():
        for x in valid_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            ys.append(x['y'].numpy())
            outs.append(out.detach().cpu().numpy())
    
    outs = np.concatenate(outs, axis=0)
    ys = np.concatenate(ys, axis=0)
    
    outs = target_scaler.inverse_transform(outs)
    ys = target_scaler.inverse_transform(ys)
    
    m = metric(outs, ys)
    print(epoch, m)
    if m > best_metric:
        torch.save(model.state_dict(), 'model4.pth')
        best_metric = m
    model.train()
    optimizer.zero_grad()

0 0.5205314009661836
1 0.535024154589372
2 0.44323671497584544
3 0.5193236714975845
4 0.5398550724637681


KeyboardInterrupt: 

In [65]:
df_test_static['TST'] = 0
df_test_static['C'] = 0
test_ds = EvrazDatasetV(df_test, df_test_static)
test_dl = DataLoader(test_ds, shuffle=False, batch_size=1, num_workers=0)

for i in range(5):
    model.load_state_dict(torch.load('model'+str(i) + '.pth'))
    model.eval()
    outs = []
    with torch.no_grad():
        for x in test_dl:
            out = model(x["static_cat"].long().to(device),
              x['static_numeric'].float().to(device),
              x['ts_cat'].long().to(device),
              x['ts_numeric'].float().to(device)
            )
            outs.append(out.detach().cpu().numpy())
    outs = np.concatenate(outs, axis=0)
    outs = target_scaler.inverse_transform(outs)
    df_test_static['TST'] += outs[:,0] / 5.0
    df_test_static['C'] += outs[:,1] / 5.0

In [66]:
q = df_test_static[['NPLV','TST','C']].copy()
q.to_csv('sub2.csv',index=False)

In [67]:
q

,NPLV,TST,C
0,512324,1651.794128,0.054295
1,512327,1648.189636,0.058045
2,512328,1648.035522,0.058825
3,512331,1648.213440,0.058173
4,512333,1648.038147,0.058030
...,...,...,...
775,513369,1649.587830,0.057576
776,513370,1649.741425,0.057481
777,513371,1649.352600,0.057496
778,513372,1649.949036,0.057389


In [61]:
q

,NPLV,TST,C
0,512324,1654.231567,0.047168
1,512327,1652.980286,0.055645
2,512328,1651.970215,0.058765
3,512331,1652.762573,0.056714
4,512333,1652.773438,0.056729
...,...,...,...
775,513369,1654.987061,0.055939
776,513370,1654.961182,0.055748
777,513371,1654.581360,0.055791
778,513372,1655.233032,0.055838
